In [2]:
import pickle

In [4]:
with open('../Data/allplayerFantasyGameLogs.pickle', 'rb') as handle:
  allplayerFantasyGameLogs = pickle.load(handle)

In [5]:
allplayerFantasyGameLogs = allplayerFantasyGameLogs.set_index('GAME_DATE')

##Helper functions to do EDA and feature extraction

function to generate feature tables, namely training and testing data sets given a game date, where training data set contains all information before the predefined game date

In [6]:
def get_train_test(previous_date ,new_game_date): #format like'2/10/2016'
    train = allplayerFantasyGameLogs.ix[:previous_date]
    test = allplayerFantasyGameLogs.ix[new_game_date]
    return(train, test)

In [7]:
train, test = get_train_test('12/10/2015','12/11/2015')

In [8]:
train.head()

,SEASON_ID,Player_ID,Game_ID,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,...,BLK,TOV,PF,PTS,PLUS_MINUS,DouBL,TriBL,FanPTs,fullName,position1
GAME_DATE,,,,,,,,,,,,,,,,,,,,,
2015-10-27,22015,101126,0021500003,NOP @ GSW,L,19,0,1,0.000,0,...,0,0,5,0,-21,False,False,6.50,Nate Robinson,PG
2015-10-27,22015,202389,0021500002,CLE @ CHI,L,21,2,6,0.333,0,...,2,3,2,5,-1,False,False,16.75,Timofey Mozgov,C
2015-10-27,22015,202087,0021500003,NOP @ GSW,L,27,4,5,0.800,0,...,0,3,3,8,-8,False,False,20.25,Alonzo Gee,SF
2015-10-27,22015,2581,0021500001,DET @ ATL,W,16,1,6,0.167,1,...,0,3,1,3,-14,False,False,10.00,Steve Blake,PG
2015-10-27,22015,202710,0021500002,CHI vs. CLE,W,37,6,14,0.429,0,...,1,0,1,17,3,False,False,32.25,Jimmy Butler,SG


function to aggregate statistics for each player up to a given game date

In [9]:
def aggr_stats(date):
    interest_columns = ['fullName','Player_ID','MIN','PTS','FGM','FGA', 'FG3M','FG3A', \
                        'REB','AST','STL','TOV','PLUS_MINUS','DouBL','TriBL','FanPTs']
    tmp = allplayerFantasyGameLogs.ix['10/27/2015':date]
    tmp.grouped = tmp[interest_columns].reset_index().groupby('fullName')
    
    def aggr(group):
        group['NumGames'] = group['fullName'].count()
        group['AvgFanPTs'] = group['FanPTs'].mean()
        group['AvgPTS'] = group['PTS'].mean()
        group['AvgMIN'] = group['MIN'].mean() 
        group['AvgFGM'] = group['FGM'].mean()
        group['AvgFGA'] = group['FGA'].mean()
        group['AvgFG3M'] = group['FG3M'].mean()
        group['AvgFG3A'] = group['FG3A'].mean()
        group['AvgREB'] = group['REB'].mean()
        group['AvgAST'] = group['AST'].mean()
        group['AvgSTL'] = group['STL'].mean()
        group['AvgTOV'] = group['TOV'].mean()
        group['AvgPLUS_MINUS'] = group['PLUS_MINUS'].mean()
        group['NumDouBL'] = group['DouBL'].sum()
        group['NumTriBL'] = group['TriBL'].sum()

        return group
    
    Newdf = tmp.grouped.apply(aggr)
    Newdf = Newdf.drop(['GAME_DATE','MIN','PTS','FGM','FGA', 'FG3M','FG3A','REB','AST','STL','TOV','PLUS_MINUS','DouBL','TriBL','FanPTs'],axis=1)
    
    return(Newdf)

In [10]:
PlayerFeaturesTable_02_29_2016 = aggr_stats('2/29/2016')

In [11]:
PlayerFeaturesTable_02_29_2016.head()

,fullName,Player_ID,NumGames,AvgFanPTs,AvgPTS,AvgMIN,AvgFGM,AvgFGA,AvgFG3M,AvgFG3A,AvgREB,AvgAST,AvgSTL,AvgTOV,AvgPLUS_MINUS,NumDouBL,NumTriBL
0,Nate Robinson,101126,2,4.000000,0.000000,11.500000,0.000000,0.500000,0.000000,0.500000,0.000000,2.000000,0.500000,0.000000,-10.000000,0,0
1,Timofey Mozgov,202389,56,14.223214,6.357143,17.821429,2.732143,4.982143,0.017857,0.125000,4.500000,0.410714,0.285714,0.964286,0.214286,1,0
2,Alonzo Gee,202087,58,11.012931,3.827586,21.448276,1.534483,3.172414,0.258621,0.913793,3.051724,0.948276,0.896552,0.637931,-2.517241,0,0
3,Steve Blake,2581,36,11.430556,4.277778,16.000000,1.527778,3.944444,0.944444,2.611111,1.444444,3.083333,0.416667,1.500000,-1.138889,0,0
4,Jimmy Butler,202710,48,39.682292,22.375000,37.812500,7.437500,16.250000,1.104167,3.333333,5.187500,4.270833,1.687500,1.979167,0.187500,6,0


In [ ]:
with open('../Data/PlayerFeaturesTable_02_29_2016.pickle', 'wb') as handle:
  pickle.dump(PlayerFeaturesTable_02_29_2016,handle)

function to generate a score(PTS) leader board given a game date, the same applies to other 
attributes, such as 'REB', 'AST', 'FG3', 'BLK', 'PLUS_MINUS'

In [12]:
def get_leader_board(attr, date):
    tmp = allplayerFantasyGameLogs.ix['10/27/2015':date]
    tmp = tmp[['fullName',attr]].reset_index().groupby('fullName').mean()
    result_df = tmp.sort_values([attr], ascending=False)

    return(result_df)

In [13]:
#Test 1. leader board for scoring
test1 = get_leader_board('PTS','2/29/2016')
test1.head()

,PTS
fullName,
Stephen Curry,30.678571
James Harden,28.500000
Kevin Durant,27.905660
DeMarcus Cousins,27.306122
Damian Lillard,25.358491


In [14]:
#Test 2. leader board for Fantasy Points
get_leader_board('FanPTs','2/29/2016').head(10)

,FanPTs
fullName,
Russell Westbrook,54.112500
Stephen Curry,53.151786
DeMarcus Cousins,51.505102
James Harden,50.995833
Kevin Durant,49.650943
LeBron James,47.350877
Anthony Davis,46.799020
John Wall,46.105932
Damian Lillard,44.198113


In [15]:
#Test 3. leader board for Fantasy Points
get_leader_board('PLUS_MINUS','2/29/2016').head(10)

,PLUS_MINUS
fullName,
Stephen Curry,14.267857
Draymond Green,13.964912
Klay Thompson,12.107143
Kawhi Leonard,9.685185
Jorge Gutierrez,9.000000
Kevin Durant,8.735849
Michael Kidd-Gilchrist,8.285714
Tim Duncan,8.113636
Russell Westbrook,8.066667


#Visualization